In [1]:
import os
import sys
sys.path.append(os.pardir)

In [2]:
import datetime
import pandas as pd
from sklearn import preprocessing
from credible import connectors

In [3]:
engine = connectors.connect_to_sqlite()

In [4]:
users = pd.read_sql_table('users', engine)

In [7]:
# derived columns to be inserted to database
df = pd.DataFrame()
df['user_id'] = users.user_id
df.head(2)

,user_id
0,l6BmjZMeQD3rDxWUbiAiow
1,4XChL029mKr5hydo79Ljxg


## EDA on Users

In [18]:
users.head(2)

,_id,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,1,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",84,17,25,5,...,0,0,0,0,1,1,1,1,2,0
1,2,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",48,22,16,4,...,0,0,0,0,0,0,1,1,0,0


In [20]:
users.columns

Index(['_id', 'user_id', 'name', 'review_count', 'yelping_since', 'friends',
       'useful', 'funny', 'cool', 'fans', 'elite', 'average_stars',
       'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note',
       'compliment_plain', 'compliment_cool', 'compliment_funny',
       'compliment_writer', 'compliment_photos'],
      dtype='object')

## Creata User Meta Information

In [16]:
df['num_of_friends'] = users.friends.apply(lambda x: len(x.split(',')))

In [17]:
df.head(2)

,user_id,num_of_friends
0,l6BmjZMeQD3rDxWUbiAiow,99
1,4XChL029mKr5hydo79Ljxg,1152


## Create Friends Mapping

In [ ]:
df_friends = pd.DataFrame()
for ind, user in users.iterrows():
    friends = user.friends.split(',')
    for friend_id in friends:
        df_friends.append({
            'user_id': user.user_id,
            'friend_id': friend_id
        }, ignore_index=True)

In [ ]:
df_friends.head(2)

## Create the Table

### meta table

In [22]:
df.to_sql(name='users_meta', con=engine, index=False)

In [23]:
pd.read_sql_table('users_meta', engine).head(2)

,user_id,num_of_friends
0,l6BmjZMeQD3rDxWUbiAiow,99
1,4XChL029mKr5hydo79Ljxg,1152


### friends table

In [22]:
df.to_sql(name='friends', con=engine, index=False)

In [23]:
pd.read_sql_table('friends', engine).head(2)

,user_id,num_of_friends
0,l6BmjZMeQD3rDxWUbiAiow,99
1,4XChL029mKr5hydo79Ljxg,1152
